### Adapting Assembly dataset to OadTR code

In [2]:
import pickle 
import json 

# Load the data
with open('/home/aleflabo/ego_procedural/OadTR/data/thumos_test_anno.pickle', 'rb') as f:
    anno = pickle.load(f)
with open('/home/aleflabo/ego_procedural/OadTR/data/thumos_all_feature_test_tsn_v2.pickle', 'rb') as f:
    data = pickle.load(f)


In [4]:
path_to_mapping = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/mapping.txt'
to_eliminate = ['nusar-2021_action_both_9026-b04b_9026_user_id_2021-02-03_163855',
                'nusar-2021_action_both_9061-c13d_9061_user_id_2021-02-09_143830',
                'nusar-2021_action_both_9034-c08b_9034_user_id_2021-02-23_175357',
                'nusar-2021_action_both_9054-c01a_9054_user_id_2021-02-08_154802',
                'nusar-2021_action_both_9024-c12c_9024_user_id_2021-02-23_153804',
                'nusar-2021_action_both_9024-c04c_9024_user_id_2021-02-03_142724',
                'nusar-2021_action_both_9015-c10c_9015_user_id_2021-02-02_165955',
                'nusar-2021_action_both_9074-a03_9074_user_id_2021-02-11_151600',
                'nusar-2021_action_both_9014-c01b_9014_user_id_2021-02-02_150612',
                'nusar-2021_action_both_9026-b08a_9026_user_id_2021-02-03_165029',
                'nusar-2021_action_both_9024-c09b_9024_user_id_2021-02-03_143716'
                ]


def load_action_mapping(map_fname):
    label2index = dict()
    index2label = dict()
    with open(map_fname, 'r') as f:
        content = f.read().split('\n')[0:-1]
        for line in content:
            label2index[line.split()[1]] = int(line.split()[0])
            index2label[int(line.split()[0])] = line.split()[1]

    return label2index, index2label

def load_split(map_fname):
    file_name = []
    with open(map_fname, 'r') as f:
        content = f.read().split('\n')[0:-1]
        for line in content:
            if not line.replace('.csv', '') in to_eliminate:
                file_name.append(line.replace('.csv', ''))

    return file_name

assembly = {}
assembly["class_index"] = list(load_action_mapping(path_to_mapping)[0].keys())

assembly["train_session_set"] = load_split('/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/split.train')
assembly["test_session_set"] = load_split('/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/split.test')

# read json file    
with open('/home/aleflabo/ego_procedural/OadTR/data/data_info_new.json', 'r') as outfile:
    data_json = json.load(outfile)

data_json['assembly'] = assembly

# save json file
with open('/home/aleflabo/ego_procedural/OadTR/data/data_info_new.json', 'w') as outfile:
    json.dump(data_json, outfile)

In [5]:
from glob import glob
import numpy as np

def load_action_mapping(map_fname):
    label2index = dict()
    index2label = dict()
    with open(map_fname, 'r') as f:
        content = f.read().split('\n')[0:-1]
        for line in content:
            label2index[line.split()[1]] = int(line.split()[0])
            index2label[int(line.split()[0])] = line.split()[1]

    return label2index, index2label

# label2index, index2label = load_action_mapping('/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/mapping.txt')

label2index, index2label = load_action_mapping('/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/mapping.txt')

files = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/groundTruth/*.txt'
# files = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/groundTruth_frames_BG_train/*.txt'
files = glob(files)

assembly['labels_train'] = {}
assembly['labels_test'] = {}
assembly['labels'] = {}
for fname in files:
    if fname.replace('.txt', '').split('/')[-1] in assembly["train_session_set"]:
        suffix = 'train'
    else: 
        suffix = 'test'
    with open(fname) as fp:
        labels = fp.read().split('\n')[:-1]
    gt = []
    for label in labels:
        zeros = np.zeros(len(assembly["class_index"]))
        
        try:    
            zeros[label2index[label]] = 1
        except:
            zeros[label2index[label.split('-')[0] + '-' + label.split('-')[2] + '-' + label.split('-')[1]]] = 1

        gt.append(zeros.reshape(1, -1))
    gt = np.concatenate(gt, axis=0)

    assembly['labels_{}'.format(suffix)][fname.replace('.txt', '').split('/')[-1]] = {}
    assembly['labels_{}'.format(suffix)][fname.replace('.txt', '').split('/')[-1]]['anno'] = gt
    assembly['labels_{}'.format(suffix)][fname.replace('.txt', '').split('/')[-1]]['feature_length'] = gt.shape[0]
    
# save pickle file
with open('/home/aleflabo/ego_procedural/OadTR/data/assembly/assembly_test_anno.pickle', 'wb') as outfile:
    pickle.dump(assembly['labels_test'], outfile)
    
# save pickle file
with open('/home/aleflabo/ego_procedural/OadTR/data/assembly/assembly_train_anno.pickle', 'wb') as outfile:
    pickle.dump(assembly['labels_train'], outfile)

In [4]:
VIEWS = ['HMC_21110305_mono10bit', 'HMC_84355350_mono10bit', 'HMC_21179183_mono10bit', 'HMC_84358933_mono10bit']
feature_dir = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/assembly101/data/TSM_features/'


In [5]:
import pandas as pd
def _load_assembly_features(file_name, view, env):

    path = '/home/aleflabo/ego_procedural/CVPR22-POC/dataset/Assembly/assembly101/data/annotations/mistake-annotations/mistake_labels/'
    mistake_labels = pd.read_csv(path+file_name+'.csv')
    
    st_frame = mistake_labels['start_frame'].values[0]
    end_frame = mistake_labels['end_frame'].values[-1]
    
    elements = []
    with env[view].begin() as e:
        for i in range(st_frame, end_frame):
            key = file_name.replace('.csv', '').split('/')[-1] + frames_format.format(view, view, i)
            data = e.get(key.strip().encode('utf-8'))
            if data is None:
                print('no available data: {} {}'.format(file_name, view))
                return -1*np.ones((end_frame-st_frame,2048))
                # exit(2)
            data = np.frombuffer(data, 'float32')
            assert data.shape[0] == 2048
            elements.append(data)

    feature = np.array(elements)
        
    return feature


In [7]:
import lmdb
#don't print warnings
import warnings
warnings.filterwarnings("ignore")

with open('/home/aleflabo/ego_procedural/CVPR22-POC/statistic_input.pkl', 'rb') as f:
    statistic = pickle.load(f)
env = {view: lmdb.open(f'{feature_dir}/{view}', readonly=True, lock=False) for view in VIEWS}
frames_format = "/{}/{}_{:010d}.jpg"
assembly['train_feats'] = {}
assembly['test_feats'] = {}
split='test'
for video in assembly["{}_session_set".format(split)]:

    info = statistic[video.replace('.csv', '').split('/')[-1]]
    actual_views = list(set(VIEWS).intersection(set(info.keys())))
    if len(actual_views) == 0:
        print('no available data: {}'.format(video))
        continue
    for view in actual_views: 
        assembly['{}_feats'.format(split)][video] = {}
        assembly['{}_feats'.format(split)][video]['rgb'] = {}
        assembly['{}_feats'.format(split)][video]['flow'] = {}

        feats = _load_assembly_features(video, view, env)
        if (feats != -1*np.ones((feats.shape[0], 2048))).all():
            assembly['{}_feats'.format(split)][video]['rgb'] = feats
            assembly['{}_feats'.format(split)][video]['flow'] = feats
            # video = view + '-' + old_video
            # new_video_list.append(video)
            # features[video] = feats
            if feats.shape[0] > 10000:
                print(video,feats.shape)
        else:
            print('skipping video', video)
            continue

    
with open('/home/aleflabo/ego_procedural/OadTR/data/assembly/assembly_all_feature_{}_tsn_v2.pickle'.format(split), 'wb') as outfile:
    pickle.dump(assembly['{}_feats'.format(split)], outfile)

nusar-2021_action_both_9053-c12e_9053_user_id_2021-02-08_142744 (10608, 2048)
nusar-2021_action_both_9053-c12e_9053_user_id_2021-02-08_142744 (10608, 2048)
nusar-2021_action_both_9054-a14_9054_user_id_2021-02-08_152449 (10010, 2048)
nusar-2021_action_both_9054-a14_9054_user_id_2021-02-08_152449 (10010, 2048)
nusar-2021_action_both_9063-c02b_9063_user_id_2021-02-17_094025 (11783, 2048)
nusar-2021_action_both_9063-c02b_9063_user_id_2021-02-17_094025 (11783, 2048)
nusar-2021_action_both_9045-a23_9045_user_id_2021-02-05_165228 (23831, 2048)
nusar-2021_action_both_9045-a23_9045_user_id_2021-02-05_165228 (23831, 2048)
nusar-2021_action_both_9076-c13d_9076_user_id_2021-02-12_115510 (13307, 2048)
nusar-2021_action_both_9076-c13d_9076_user_id_2021-02-12_115510 (13307, 2048)
nusar-2021_action_both_9052-c12c_9052_user_id_2021-02-08_114847 (13278, 2048)
nusar-2021_action_both_9052-c12c_9052_user_id_2021-02-08_114847 (13278, 2048)
nusar-2021_action_both_9033-a30_9033_user_id_2021-02-04_132852 (1783

### Postprocessing for Anomaly Detection

In [1]:
import pickle
dec_step = 0
with open('/home/aleflabo/ego_procedural/OadTR/models/en_3_decoder_5_lr_drop_1/results/results_dec_test_0.pickle'.format(str(dec_step)), 'rb') as f:
    result = pickle.load(f)


In [5]:
result['labels'], result['probs']

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[2.9828477e-08, 3.1965872e-08, 2.7996728e-08, ..., 7.1956689e-04,
         8.1203989e-04, 8.5383432e-04],
        [2.5304249e-07, 2.7841472e-07, 2.4012058e-07, ..., 4.9408929e-05,
         4.9253431e-05, 5.2421041e-05],
        [2.5303839e-07, 2.7840969e-07, 2.4011624e-07, ..., 4.9413546e-05,
         4.9257851e-05, 5.2425887e-05],
        ...,
        [8.4331893e-09, 9.4674233e-09, 8.3222753e-09, ..., 1.6505046e-05,
         1.6651491e-05, 1.7322502e-05],
        [4.8670677e-07, 5.4227286e-07, 4.7270620e-07, ..., 1.7691648e-03,
         1.5160488e-03, 1.7814003e-03],
        [2.5304226e-07, 2.7841446e-07, 2.4012013e-07, ..., 4.9409449e-05,
         4.9253951e-05, 5.2421590e-05]], dtype=float32))